### Problem 6.4

Let us define our domain: 

In [3]:
import numpy as np 
import matplotlib.pyplot as plt 

# Domain parameters
L  = 100 
dx = 0.1 

# 1D Array holding grid points, x
x = np.arange(0, L+dx, dx)  

# Number of grid points in x
N = len(x) 

# Velocity (v) and stress (sigma)
v     = np.zeros(N)
sigma = np.zeros(N)

# Material properties - this is an array in so 
# properties can be defined at each point in the grid
rho = np.zeros(N)
mu  = np.zeros(N)

Define the initial condition as per the problem: 

In [7]:
def apply_initial_condition(x, v, sigma): 
    # Equation 6.44
    sigma[:] = 0
    v = -0.2 * (x-50) * np.exp(-0.1*(x-50)**2)
    return v, sigma

### Solver:

The point of these methods is that they are flexible to whatever medium the wavefield is being simulated in. That is, the scheme is independent of whether the domain is homogeneous (part A of the problem) or heterogeneous (part B). Hence, we want to define a function that solves for the next timestep, which is relatively generic. 

At each grid point, the solver computes the new velocity and stress based on values from adjacent grid points. This can be written efficiently using arrays. Let us store the stress and velocity at each grid point in 1D arrays. The stress in the first grid point (j=1) will be stored in $\sigma$[0], and the stress in the second grid point in $\sigma[1]$ and so on.... Note here at Python indexing starts at 0 (like most programming languages, e.g. C++, Rust etc, while some others such as MATLAB start at 1!). If there are N grid points then the final stress value will be stored in $\sigma[N-1]$. 

Let us now consider the first point (away from the boundary) that we could apply our solver to, at the second (j=2) grid point. To update the velocity we know this would be: 

$$
v^{n+1}_{j=2} = v^{n-1}_{j=2} + \frac{\Delta t}{\Delta x \,\rho_{j=2}} \left( \sigma_{j=3}^n  - \sigma^n_{j=1}\right)
$$

For now let us assume that we have access to the term $v^{n-1}_{j=2}$ which will be stored in a separate array, as well as the simulation parameters $\Delta t$, $\Delta x$. The important part to note is that we need access to the term $ \sigma_{j=3}^n  - \sigma^n_{j=1}$. Similarly, for the following $j$ values we need access to: 


$$
j = 2 : \quad  \sigma_{j=3}^n  - \sigma^n_{j=1}\\
j = 3 : \quad  \sigma_{j=4}^n  - \sigma^n_{j=2}\\
j = 4 : \quad  \sigma_{j=5}^n  - \sigma^n_{j=3}\\
j = 5 : \quad  \sigma_{j=6}^n  - \sigma^n_{j=4}
$$

We may compute all of these at the same time, if we take two sub-arrays of $\sigma$ and subtract one from the other. The first array starts at the 3rd grid point (j=3) and represents the $\sigma^n_{j+1}$ values, while the second array starts at j=1 and represnts the $\sigma^n_{j+1}$ values. 

The point here is that for EACH point (excluding the boundary terms) we need to compute the difference of the terms either side of it, which can be done as an array subtraction, rather than in a loop where the computation must be done for each individual grid point in serial. 

To do this we use slicing, including negative slicing. Read more about this [here](https://www.w3schools.com/python/numpy/numpy_array_slicing.asp). 

In [5]:
def step_in_time(s_old, s, v_old, v, dx, dt, mu, rho, use_Dirichlet_BC): 
    
    # Notes on slicing: 
    # [1:-1] includes all points except boundaries
    # [2:  ] is from 3rd to end 
    # [:-2 ] is from start to 3rd-last (missing last two points)
    
    # [1:-1] represents lower index = j 
    # [2:  ] represents lower index = j + 1 
    # [:-2 ] represents lower index = j - 1
    
    # Variable names (s and v): 
    # s for sigma, v for v (velocity)
    # ? below means s or v
    # ?_new  represents superscript = n + 1 
    # ?      represents superscript = n 
    # ?_old  represents superscript = n - 1
    
    # Compute new v and s 
    v_new[1:-1] = v_old[1:-1] + (dt/(rho[1:-1]*dx)) * (s[2:] - s[:-2]) 
    s_new[1:-1] = s_old[1:-1] + (dt*mu[1:-1]/dx) * (v[2:] - v[:-2])
    
    NOTE AT THE BOUNDARY WE WILL NEED TO USE A LOWER ORDER SCHEME (FIRST ORDER, FORWARD OR BACKWARD)
    APPLY THE BOUNDARY CONDITIONS CORRRECTLY 
    # Apply boundary conditions: 
    if use_Dirichlet_BC: 
        # Dirichlet: 
        v_new[0]  = 0 
        v_new[-1] = 0 
    else: 
        s_new[0]  = 0
        s_new[-1] = 0
    return v_new, s_new 

SyntaxError: invalid syntax (4231033743.py, line 23)